<a href="https://colab.research.google.com/github/UTKARSH4/bbc_news_archive/blob/main/bbc_news_archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import csv
import pandas as pd


In [171]:
## meta data
data=[]
with open('bbc-news-data.csv','r') as f:
  for row in csv.reader(f):
    data.append(row[0].split('\t'))


## content of data
data_cont=[]
with open('bbc-news-data.csv','r') as f:
  for row in csv.reader(f):
    data_cont.append(row[1:])   

col=data[0]
df = pd.DataFrame(data[1:],columns=col) 


for i in range(len(data)):
  data_cont[i].insert(0, data[i][-1])
    


In [172]:
del df['content']
df['content_all']=data_cont[1:]
#df=df['category','file','title','content_all']


In [179]:
df['merged']=''
for i in range(df.shape[0]):
  df.merged[i] = ' '.join(df.iloc[i][3])

In [183]:
df.merged[0]


0        Quarterly profits at US media giant TimeWarne...
1        The owners of embattled Russian oil giant Yuk...
2        British Airways has blamed high fuel prices f...
3        Shares in UK drinks and food firm Allied Dome...
4        Japan's economy teetered on the brink of a te...
                              ...                        
1618     Internet TV has been talked about since the s...
1619     Thousands of products and tens of thousands o...
1620     Making games for future consoles will require...
1621     A new European directive could put software w...
1622     The man making sure US computer networks are ...
Name: merged, Length: 1623, dtype: object

In [115]:

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]


In [236]:
vocab_size = 100000
embedding_dim = 16
max_length = 1000 # Expected max_len is 120 a few cells down.
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [233]:
training_portion=0.8
train_set=df.iloc[0:int(df.shape[0]*training_portion)]
eval_set=df.iloc[int(df.shape[0]*training_portion)+1:-1]

In [235]:
tokenizer_text =Tokenizer(oov_token=oov_tok)
tokenizer_text.fit_on_texts(df.merged)
word_index = tokenizer_text.word_index

train_sequences = tokenizer_text.texts_to_sequences(train_set.merged)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


eval_sequences = tokenizer_text.texts_to_sequences(eval_set.merged)
eval_padded = pad_sequences(eval_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [239]:
tokenizer_label =Tokenizer()
tokenizer_label.fit_on_texts(df.category)
label_index = tokenizer_text.word_index

train_labels = np.array(tokenizer_label.texts_to_sequences(train_set.category))


eval_labels = np.array(tokenizer_label.texts_to_sequences(eval_set.category))


In [237]:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 16)          1600000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 150       
Total params: 1,600,558
Trainable params: 1,600,558
Non-trainable params: 0
_________________________________________________________________


In [240]:
num_epochs = 30
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(eval_padded, eval_labels))

Epoch 1/30
41/41 [==============================] - 2s 23ms/step - loss: 1.7715 - accuracy: 0.3156 - val_loss: 1.8720 - val_accuracy: 0.2229
Epoch 2/30
41/41 [==============================] - 1s 20ms/step - loss: 1.6654 - accuracy: 0.3961 - val_loss: 2.1221 - val_accuracy: 0.2229
Epoch 3/30
41/41 [==============================] - 1s 20ms/step - loss: 1.5195 - accuracy: 0.3841 - val_loss: 2.6055 - val_accuracy: 0.2786
Epoch 4/30
41/41 [==============================] - 1s 20ms/step - loss: 1.4035 - accuracy: 0.4239 - val_loss: 3.1043 - val_accuracy: 0.2663
Epoch 5/30
41/41 [==============================] - 1s 21ms/step - loss: 1.3307 - accuracy: 0.4727 - val_loss: 3.4820 - val_accuracy: 0.2601
Epoch 6/30
41/41 [==============================] - 1s 20ms/step - loss: 1.2765 - accuracy: 0.5278 - val_loss: 3.7706 - val_accuracy: 0.2601
Epoch 7/30
41/41 [==============================] - 1s 20ms/step - loss: 1.2029 - accuracy: 0.5732 - val_loss: 4.0094 - val_accuracy: 0.2508
Epoch 8/30
41